In [48]:
from esa_snappy import ProductIO, HashMap, GPF, Product, ProductData

In [49]:
import jpy

In [50]:
Integer = jpy.get_type("java.lang.Integer")
HashMap = jpy.get_type("java.util.HashMap")

In [51]:
import matplotlib.pyplot as plt
import numpy as np
import os
import subprocess
import glob


In [181]:
print(subprocess.Popen(['gpt','-h','Terrain-Correction'],stdout=subprocess.PIPE, universal_newlines=True).communicate()[0])


Usage:
  gpt Terrain-Correction [options] 

Description:
  RD method for orthorectification


Source Options:
  -Ssource=<file>    Sets source 'source' to <filepath>.
                     This is a mandatory source.

Parameter Options:
  -PalignToStandardGrid=<boolean>                 Force the image grid to be aligned with a specific point
                                                  Default value is 'false'.
  -PapplyRadiometricNormalization=<boolean>       Sets parameter 'applyRadiometricNormalization' to <boolean>.
                                                  Default value is 'false'.
  -PauxFile=<string>                              The auxiliary file
                                                  Value must be one of 'Latest Auxiliary File', 'Product Auxiliary File', 'External Auxiliary File'.
                                                  Default value is 'Latest Auxiliary File'.
  -PdemName=<string>                              The digital elevation model.
     

In [53]:
image1 = ProductIO.readProduct("D:\SAR\BARC\postMonsoon\S1A_IW_SLC__1SDV_20231021T010332_20231021T010400_050856_06213A_CD75.SAFE.zip")
print(list(image1.getBandNames()))

['i_IW1_VH', 'q_IW1_VH', 'Intensity_IW1_VH', 'i_IW1_VV', 'q_IW1_VV', 'Intensity_IW1_VV', 'i_IW2_VH', 'q_IW2_VH', 'Intensity_IW2_VH', 'i_IW2_VV', 'q_IW2_VV', 'Intensity_IW2_VV', 'i_IW3_VH', 'q_IW3_VH', 'Intensity_IW3_VH', 'i_IW3_VV', 'q_IW3_VV', 'Intensity_IW3_VV']


In [54]:
image2 = ProductIO.readProduct("D:\SAR\BARC\postMonsoon\S1A_IW_SLC__1SDV_20231102T010332_20231102T010359_051031_062740_1C6B.SAFE.zip")
print(list(image2.getBandNames()))

['i_IW1_VH', 'q_IW1_VH', 'Intensity_IW1_VH', 'i_IW1_VV', 'q_IW1_VV', 'Intensity_IW1_VV', 'i_IW2_VH', 'q_IW2_VH', 'Intensity_IW2_VH', 'i_IW2_VV', 'q_IW2_VV', 'Intensity_IW2_VV', 'i_IW3_VH', 'q_IW3_VH', 'Intensity_IW3_VH', 'i_IW3_VV', 'q_IW3_VV', 'Intensity_IW3_VV']


In [55]:
def load_sar_data(file_path):
    return ProductIO.readProduct(file_path)

In [56]:
def apply_operator(operator_name, parameters, *sources):
    params = HashMap()
    for key, value in parameters.items():
        params.put(key, value)
    return GPF.createProduct(operator_name, params, list(sources))

In [57]:
# BARC AOI coordinates (WKT Polygon)
barc_aoi = 'POLYGON((72.930559 19.005872, 72.912912 19.005872, 72.912912 19.023823, 72.930559 19.023823, 72.930559 19.005872))'

In [58]:
pre_event = load_sar_data("D:\SAR\BARC\postMonsoon\S1A_IW_SLC__1SDV_20231021T010332_20231021T010400_050856_06213A_CD75.SAFE.zip")
post_event = load_sar_data("D:\SAR\BARC\postMonsoon\S1A_IW_SLC__1SDV_20231102T010332_20231102T010359_051031_062740_1C6B.SAFE.zip")

## Apply Orbit File

In [59]:
params_orbit = {'orbitType': 'Sentinel Precise (Auto Download)'}
pre_event = apply_operator('Apply-Orbit-File', params_orbit, pre_event)
post_event = apply_operator('Apply-Orbit-File', params_orbit, post_event)

## TOPSAR SPLIT

In [60]:
# Only needed if AOI spans multiple subswaths (BARC is in IW2)
params_split = {'subswath': 'IW2', 'selectedPolarisations': 'VV'}
pre_event_split = apply_operator('TOPSAR-Split', params_split, pre_event)
post_event_split = apply_operator('TOPSAR-Split', params_split, post_event)

## Coregisteration and ESD

In [61]:
params_coreg = {
    'initialOffsetMethod': 'Orbit',  # Use orbit for initial offset
    'esdThreshold': 0.25,  # Enable ESD for fine coregistration
    'demName': 'SRTM 1Sec HGT'
}
coregistered = apply_operator('Back-Geocoding', params_coreg, pre_event_split, post_event_split)

## ESD

In [62]:
esd = apply_operator('Enhanced-Spectral-Diversity',{},coregistered)

## Interferogram Generation

In [63]:
params_interferogram = {
    'subtractFlatEarthPhase': True,
    'degreeOfFreedom': 4  # For Goldstein filter later
}
interferogram = apply_operator('Interferogram', params_interferogram, esd)

In [64]:
print(f"Product Name: {interferogram.getName()}")
print(f"Product Width: {interferogram.getSceneRasterWidth()}")
print(f"Product Height: {interferogram.getSceneRasterHeight()}")
print(f"Product Bands: {list(interferogram.getBandNames())}")


Product Name: S1A_IW_SLC__1SDV_20231102T010332_20231102T010359_051031_062740_1C6B_Orb_Stack_Ifg
Product Width: 25482
Product Height: 13581
Product Bands: ['i_ifg_IW2_VV_02Nov2023_21Oct2023', 'q_ifg_IW2_VV_02Nov2023_21Oct2023', 'Intensity_ifg_IW2_VV_02Nov2023_21Oct2023', 'Phase_ifg_IW2_VV_02Nov2023_21Oct2023', 'coh_IW2_VV_02Nov2023_21Oct2023']


## TOPSAR DEBURST

In [65]:
deburst = apply_operator('TOPSAR-Deburst', {}, interferogram)
list(deburst.getBandNames())

['i_ifg_IW2_VV_02Nov2023_21Oct2023',
 'q_ifg_IW2_VV_02Nov2023_21Oct2023',
 'Intensity_ifg_IW2_VV_02Nov2023_21Oct2023',
 'Phase_ifg_IW2_VV_02Nov2023_21Oct2023',
 'coh_IW2_VV_02Nov2023_21Oct2023']

## Topographic Phase Removal

In [66]:
params_topo = {
    'demName': 'SRTM 1Sec HGT'
}
topo_removed = apply_operator('TopoPhaseRemoval', params_topo, deburst)
list(topo_removed.getBandNames())

['i_ifg_VV_02Nov2023_21Oct2023',
 'q_ifg_VV_02Nov2023_21Oct2023',
 'Intensity_ifg_VV_02Nov2023_21Oct2023_ifg_srd_VV_02Nov2023_21Oct2023',
 'Phase_ifg_srd_VV_02Nov2023_21Oct2023',
 'coh_IW2_VV_02Nov2023_21Oct2023']

## Multilooking

In [67]:
params_multilook = {
    'nRgLooks': Integer(2),  # Range looks
    'nAzLooks': Integer(2)   # Azimuth looks
}
multilooked = apply_operator('Multilook', params_multilook, topo_removed)
list(multilooked.getBandNames())

['i_ifg_VV_02Nov2023_21Oct2023',
 'q_ifg_VV_02Nov2023_21Oct2023',
 'coh_IW2_VV_02Nov2023_21Oct2023',
 'Intensity_ifg_VV_02Nov2023_21Oct2023_ifg_srd_VV_02Nov2023_21Oct2023',
 'Phase_ifg_srd_VV_02Nov2023_21Oct2023']

## Goldstein Filtering

In [68]:
params_goldstein = {
    'alpha': 0.5,  # Filter strength
    'windowSize': 32
}
filtered = apply_operator('GoldsteinPhaseFiltering', params_goldstein, multilooked)

In [69]:
list(filtered.getBandNames())

['i_ifg_VV_02Nov2023_21Oct2023',
 'q_ifg_VV_02Nov2023_21Oct2023',
 'Intensity_ifg_VV_02Nov2023_21Oct2023_db',
 'Phase_ifg_VV_02Nov2023_21Oct2023',
 'coh_IW2_VV_02Nov2023_21Oct2023']

 ## Subsetting AOI

In [70]:
subset_params = {
    'geoRegion': barc_aoi,       # WKT polygon of AOI
    'copyMetadata': True,        # Preserve metadata (critical for InSAR)
    'fullSwath': False,          # Crop strictly to AOI (no extra pixels)
}

In [71]:
subset = apply_operator('Subset',subset_params, filtered)

In [72]:
#ProductIO.writeProduct(subset, os.path.join(r"D:\SAR\BARC\subset",'subset.dim'),'BEAM-DIMAP')

In [73]:
subset_disk = ProductIO.readProduct("D:\SAR\BARC\subset\subset.dim")

## SNAPHU Export

In [152]:
export_params = {
    'numberOfTileCols': Integer(5),
    'numberOfTileRows': Integer(5),# Increase the tile size to 500 pixels
    'colOverlap': Integer(20),       # Overlap of 50 pixels in columns
    'rowOverlap': Integer(20),       # Overlap of 50 pixels in rows
    'initMethod': 'MST',
    'statCostMode': 'DEFO',
    'targetFolder': r"D:\SAR\BARC\trial",
    'useCoherence' : True,
    'cohFile' : True
}

snaphu_export = apply_operator('SnaphuExport', export_params, subset_disk)
ProductIO.writeProduct(snaphu_export, r'D:\SAR\BARC\TTrial','snaphu')

In [153]:
ProductIO.writeProduct(snaphu_export,os.path.join( r'D:\SAR\BARC\TTrial','TTrial.dim'),'BEAM-DIMAP')

In [154]:
Product_ = ProductIO.readProduct("D:\SAR\BARC\TTrial\TTrial.dim")


In [155]:
list(Product_.getBandNames())

['Phase_ifg_VV_02Nov2023_21Oct2023', 'coh_IW2_VV_02Nov2023_21Oct2023']

## SNAPHU Unwrapping

In [156]:
width = Product_.getSceneRasterWidth()
width

282

In [157]:
tempFolder = "D:\SAR\BARC\TTrial"

In [158]:
os.chdir (tempFolder)

In [159]:
snaphu_dir = r"C:\Users\gunda\.snap\auxdata\snaphu-v2.0.4_win64\bin\snaphu.exe"

In [160]:
os.system(snaphu_dir + ' -f snaphu.conf ' + 'Phase_ifg_VV_02Nov2023_21Oct2023.snaphu.img' + ' ' + str(width))

0

## SNAPHU Import

In [175]:
Files = jpy.array('org.esa.snap.core.datamodel.Product', 2)

In [176]:
Files[0] = ProductIO.readProduct("D:\SAR\BARC\subset\subset.dim")

In [177]:
Files[1] = ProductIO.readProduct(r"D:\SAR\BARC\TTrial\UnwPhase_ifg_VV_02Nov2023_21Oct2023.snaphu.hdr")

In [178]:
HashMap     = jpy.get_type("java.util.HashMap")
params      = HashMap()
Product     = GPF.createProduct("SnaphuImport", params, Files)

In [179]:
#ProductIO.writeProduct(Product, os.path.join("D:\SAR\BARC\output",'SnaphuImport.dim'), "BEAM-DIMAP")

## Phase To Displacement and Terrain Correction

In [180]:
displacement = apply_operator("PhaseToDisplacement",{},Product)

In [182]:
params_tc = {
    "demName": "SRTM 1Sec HGT",
    "pixelSpacingInMeter": 10.0
}
tc = apply_operator('Terrain-Correction',params_tc, displacement)

In [1]:
#kmz_output = os.path.join(r"C:\Users\gunda\Desktop\subsidence", "subsidence.kmz")
#ProductIO.writeProduct(tc, kmz_output, "BEAM-DIMAP")